In [1]:
import turicreate as tc

In [2]:
speech_data = tc.SFrame('speech_emotion_recognition.csv')

Finished parsing file /home/soumya_123789/audio_project/speech_emotion_recognition.csv

Parsing completed. Parsed 100 lines in 0.11572 secs.

------------------------------------------------------
Inferred types from first 100 line(s) of file as 
column_type_hints=[float,float,float,str]
If parsing fails due to incorrect types, you can correct
the inferred type list above and pass it to read_csv in
the column_type_hints argument
------------------------------------------------------


Finished parsing file /home/soumya_123789/audio_project/speech_emotion_recognition.csv

Parsing completed. Parsed 480 lines in 0.132655 secs.

In [3]:
speech_data

mfcc,delta_1,delta_2,label
-13.68480015,0.095542379,0.012491609,a
-10.60294151,0.115995452,0.010414518,a
-15.13407898,0.006476016,-0.007506077,a
-12.95456791,0.103229165,-0.014069146,a
-12.84221935,0.1118773,0.014337501,a
-13.72909451,0.136846006,0.015095726,a
-11.96225548,0.073023356,0.008893309,a
-15.4311676,0.116335668,0.035088759,a
-13.59893417,0.143968165,0.006208563,a
-13.68214512,-0.007663798,-0.027731251,a


# train test split

In [5]:
train_data, test_set = speech_data.random_split(0.8, seed=0)

In [13]:
test_set.shuffle()

mfcc,delta_1,delta_2,label
-16.22265816,0.207132578,-0.018998325,h
-14.08856392,0.097061172,0.008959019,n
-31.92101097,0.05140027,0.047179773,n
-17.51117706,0.017613282,0.026728308,d
-10.94730663,0.086423494,0.033904098,h
-16.18053818,0.097573012,0.030995108,sa
-10.99336052,0.11638201,0.016012566,h
-31.32455063,0.069933414,0.000444407,su
-34.74782181,0.136795461,-0.004122362,n
-13.96777153,0.110571384,0.010120544,n


In [20]:
test_data, validation_data = test_set.random_split(0.7, seed=0)

# logistic regression model

In [22]:
logreg_model = tc.logistic_classifier.create(train_data, target='label',
                                             features=['mfcc', 'delta_1'],l2_penalty=0.01,
                                             max_iterations=100, validation_set= validation_data)

Logistic regression:

--------------------------------------------------------

Number of examples          : 380

Number of classes           : 7

Number of feature columns   : 2

...

| 1         | 2        | 0.003006     | 0.271053          | 0.344828            |

| 2         | 3        | 0.005087     | 0.273684          | 0.379310            |

| 3         | 4        | 0.009889     | 0.273684          | 0.379310            |

+-----------+----------+--------------+-------------------+---------------------+

SUCCESS: Optimal solution found.

In [23]:
logreg_model.training_accuracy

0.2736842105263158

In [43]:
logreg_model.validation_accuracy

0.3793103448275862

In [25]:
logreg_model.evaluate(test_data)

{'accuracy': 0.3380281690140845,
 'auc': 0.6033772245653308,
 'confusion_matrix': Columns:
 	target_label	str
 	predicted_label	str
 	count	int
 
 Rows: 14
 
 Data:
 +--------------+-----------------+-------+
 | target_label | predicted_label | count |
 +--------------+-----------------+-------+
 |      h       |        n        |   4   |
 |      h       |        h        |   1   |
 |      n       |        n        |   22  |
 |      sa      |        a        |   1   |
 |      su      |        h        |   1   |
 |      f       |        n        |   10  |
 |      sa      |        n        |   10  |
 |      d       |        n        |   4   |
 |      a       |        a        |   1   |
 |      a       |        h        |   1   |
 +--------------+-----------------+-------+
 [14 rows x 3 columns]
 Note: Only the head of the SFrame is printed.
 You can use print_rows(num_rows=m, num_columns=n) to print more rows and columns.,
 'f1_score': 0.13080975871673545,
 'log_loss': 1.8555956665358326

### training accuracy = 0.2736842105263158
### test accuracy =  0.3380281690140845
### validation accuracy = 0.3793103448275862

# decision trees

In [44]:
decision_model = tc.decision_tree_classifier.create(train_data, target='label',
                                                    features=['mfcc', 'delta_1', 'delta_2'], validation_set= validation_data,
                                                   max_depth=10)

Decision tree classifier:

--------------------------------------------------------

Number of examples          : 380

Number of classes           : 7

Number of feature columns   : 3

...

+-----------+--------------+-------------------+---------------------+-------------------+---------------------+

| Iteration | Elapsed Time | Training Accuracy | Validation Accuracy | Training Log Loss | Validation Log Loss |

+-----------+--------------+-------------------+---------------------+-------------------+---------------------+

| 1         | 0.040026     | 0.773684          | 0.241379            | 0.903280          | 1.956042            |

+-----------+--------------+-------------------+---------------------+-------------------+---------------------+

In [45]:
decision_model.training_accuracy

0.7736842105263158

In [46]:
decision_model.validation_accuracy

0.2413793103448276

In [47]:
decision_model.evaluate(test_data)

{'accuracy': 0.36619718309859156,
 'auc': 0.6421157958524132,
 'confusion_matrix': Columns:
 	target_label	str
 	predicted_label	str
 	count	int
 
 Rows: 32
 
 Data:
 +--------------+-----------------+-------+
 | target_label | predicted_label | count |
 +--------------+-----------------+-------+
 |      n       |        su       |   2   |
 |      h       |        sa       |   1   |
 |      h       |        d        |   1   |
 |      h       |        a        |   2   |
 |      f       |        sa       |   2   |
 |      n       |        h        |   1   |
 |      su      |        su       |   3   |
 |      f       |        f        |   2   |
 |      n       |        f        |   2   |
 |      sa      |        sa       |   3   |
 +--------------+-----------------+-------+
 [32 rows x 3 columns]
 Note: Only the head of the SFrame is printed.
 You can use print_rows(num_rows=m, num_columns=n) to print more rows and columns.,
 'f1_score': 0.34711884753901556,
 'log_loss': 1.846729124252033

### training accuracy = 0.7736842105263158
### test accuracy = 0.36619718309859156
### valiadation accuracy = 0.2413793103448276

# k neighbour classifier

In [48]:
nearest_model = tc.nearest_neighbor_classifier.create(train_data, target='label', features=['mfcc', 'delta_1', 'delta_2'])

Starting ball tree nearest neighbors model training.

+------------+--------------+

| Tree level | Elapsed Time |

+------------+--------------+

| 0          | 244us        |

+------------+--------------+

In [49]:
nearest_model.evaluate(test_data)

+--------------+-------------+--------------+

| Query points | % Complete. | Elapsed Time |

+--------------+-------------+--------------+

| 1            | 1.25        | 315us        |

| Done         |             | 2.841ms      |

...

+--------------+-------------+--------------+

| 1            | 1.25        | 314us        |

| Done         |             | 3.305ms      |

+--------------+-------------+--------------+

{'accuracy': 0.29577464788732394,
 'confusion_matrix': Columns:
 	target_label	str
 	predicted_label	str
 	count	int
 
 Rows: 28
 
 Data:
 +--------------+-----------------+-------+
 | target_label | predicted_label | count |
 +--------------+-----------------+-------+
 |      sa      |        n        |   4   |
 |      f       |        su       |   2   |
 |      sa      |        h        |   1   |
 |      n       |        h        |   2   |
 |      f       |        d        |   1   |
 |      n       |        sa       |   3   |
 |      h       |        a        |   3   |
 |      a       |        h        |   1   |
 |      sa      |        d        |   2   |
 |      n       |        f        |   2   |
 +--------------+-----------------+-------+
 [28 rows x 3 columns]
 Note: Only the head of the SFrame is printed.
 You can use print_rows(num_rows=m, num_columns=n) to print more rows and columns.}

In [33]:
nearest_model.predict(test_data, max_neighbors=5)

+--------------+-------------+--------------+

| Query points | % Complete. | Elapsed Time |

+--------------+-------------+--------------+

| 1            | 1.25        | 561us        |

| Done         |             | 2.549ms      |

+--------------+-------------+--------------+

dtype: str
Rows: 71
['sa', 'a', 'h', 'h', 'a', 'a', 'a', 'd', 'd', 'h', 'd', 'h', 'sa', 'sa', 'su', 'n', 'sa', 'n', 'su', 'n', 'sa', 'sa', 'h', 'su', 'a', 'a', 'n', 'd', 'n', 'n', 'n', 'sa', 'd', 'd', 'su', 'n', 'f', 'su', 'n', 'sa', 'd', 'n', 'a', 'n', 'n', 'n', 'n', 'sa', 'h', 'n', 'n', 'su', 'n', 'n', 'n', 'su', 'n', 'sa', 'n', 'sa', 'h', 'n', 'sa', 'sa', 'sa', 'n', 'sa', 'n', 'f', 'd', 'n']

### model accuaracy = 0.29577464788732394

# boosted tree classifier

In [50]:
boosted_tree_model = tc.boosted_trees_classifier.create(train_data, target='label',
                                     features= ['mfcc', 'delta_1', 'delta_2'], validation_set=validation_data,
                                                       max_depth=10, max_iterations=10)

Boosted trees classifier:

--------------------------------------------------------

Number of examples          : 380

Number of classes           : 7

Number of feature columns   : 3

...

| 3         | 0.110987     | 0.921053          | 0.241379            | 1.070816          | 1.878349            |

| 4         | 0.161842     | 0.952632          | 0.241379            | 0.895617          | 1.853230            |

| 5         | 0.208240     | 0.968421          | 0.275862            | 0.763042          | 1.865456            |

| 10        | 0.442149     | 0.994737          | 0.310345            | 0.379417          | 1.965110            |

+-----------+--------------+-------------------+---------------------+-------------------+---------------------+

In [51]:
boosted_tree_model.training_accuracy

0.9947368421052631

In [52]:
boosted_tree_model.validation_accuracy

0.3103448275862069

In [53]:
boosted_tree_model.evaluate(test_data)

{'accuracy': 0.39436619718309857,
 'auc': 0.6419873556520427,
 'confusion_matrix': Columns:
 	target_label	str
 	predicted_label	str
 	count	int
 
 Rows: 32
 
 Data:
 +--------------+-----------------+-------+
 | target_label | predicted_label | count |
 +--------------+-----------------+-------+
 |      sa      |        su       |   1   |
 |      h       |        sa       |   1   |
 |      n       |        n        |   16  |
 |      n       |        f        |   1   |
 |      d       |        f        |   1   |
 |      su      |        f        |   2   |
 |      f       |        f        |   1   |
 |      sa      |        a        |   1   |
 |      sa      |        sa       |   3   |
 |      n       |        sa       |   2   |
 +--------------+-----------------+-------+
 [32 rows x 3 columns]
 Note: Only the head of the SFrame is printed.
 You can use print_rows(num_rows=m, num_columns=n) to print more rows and columns.,
 'f1_score': 0.3264583161199703,
 'log_loss': 1.9009317296499921

### training accuracy = 0.9947368421052631
### test accuracy = 0.39436619718309857
### validation accuracy = 0.3103448275862069